# Zoom Speech Engagement Report

Speech Recognizor using https://pypi.org/project/SpeechRecognition/1.2.3/ to convert Zoom conversation into text to conduct engagement analysis. Source for transcribing long audio files from: https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python .

### Speech Recognition using mic for a single line of speech

In [12]:
import speech_recognition as sr
import pyaudio
r = sr.Recognizer()
with sr.Microphone() as source:                # use the default microphone as the audio source
    audio = r.listen(source)                   # listen for the first phrase and extract it into audio data

try:
    print("You said: " + r.recognize_google(audio))    # recognize speech using Google Speech Recognition
except LookupError:                            # speech is unintelligible
    print("Could not understand audio")

You said: Hi how are you doing today


### Speech Recognition using mic for a several lines of speech

In [33]:
def callback(recognizer, audio):                          # this is called from the background thread
    try:
        text = recognizer.recognize_google(audio)
        print("You said: " + text)  # received audio data, now need to recognize it
    except LookupError:
        print("Oops! Didn't catch that")
r = sr.Recognizer()
r.listen_in_background(sr.Microphone(), callback)

import time
while True: 
    time.sleep(0.1)    

You said: hello
You said: can you hear me why are you not working can you hear me
You said: okay cool


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/site-packages/speech_recognition/__init__.py", line 697, in threaded_listen
    if running[0]: callback(self, audio)
  File "<ipython-input-33-b0b5c40edf50>", line 3, in callback
    text = recognizer.recognize_google(audio)
  File "/anaconda3/lib/python3.7/site-packages/speech_recognition/__init__.py", line 858, in recognize_google
    if not isinstance(actual_result, dict) or len(actual_result.get("alternative", [])) == 0: raise UnknownValueError()
speech_recognition.UnknownValueError



KeyboardInterrupt: 

Code above works relatively well for speech but ouputs error and stops work when speaker pauses for a while.

### Recording transcript from Speech Recognizer

In [14]:
# Store each line of speech in a list
transcript = []

In [15]:
def callbackAndTranscribe(recognizer, audio):                          # this is called from the background thread
    try:
        line = recognizer.recognize_google(audio)
        print("You said: " + line)  # received audio data, now need to recognize it
        transcript.append(line)
    except LookupError:
        print("Oops! Didn't catch that")
r = sr.Recognizer()
r.listen_in_background(sr.Microphone(), callbackAndTranscribe)

import time
while True: 
    time.sleep(0.1)

You said: hello my name is science
You said: checking if this is working
You said: and to see if I can get a proper transcript
You said: also my name is since not science hello can you hear me
You said: my name is sense not science


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/site-packages/speech_recognition/__init__.py", line 697, in threaded_listen
    if running[0]: callback(self, audio)
  File "<ipython-input-15-c68b5cb38d70>", line 3, in callbackAndTranscribe
    line = recognizer.recognize_google(audio)
  File "/anaconda3/lib/python3.7/site-packages/speech_recognition/__init__.py", line 858, in recognize_google
    if not isinstance(actual_result, dict) or len(actual_result.get("alternative", [])) == 0: raise UnknownValueError()
speech_recognition.UnknownValueError



KeyboardInterrupt: 

In [16]:
transcript

['hello my name is science',
 'checking if this is working',
 'and to see if I can get a proper transcript',
 'also my name is since not science hello can you hear me',
 'my name is sense not science']

### Recording transcript from Audio File

Playing Audio in Jupyter Notebook using pyaudio:

In [23]:
#NB Code source: https://realpython.com/playing-and-recording-sound-python/#pyaudio
import pyaudio
import wave

filename = "SteveJobsCommencementSpeech.wav"

# Set chunk size of 1024 samples per data frame
chunk = 1024  

# Open the sound file 
wf = wave.open(filename, 'rb')

# Create an interface to PortAudio
p = pyaudio.PyAudio()

# Open a .Stream object to write the WAV file to
# 'output = True' indicates that the sound will be played rather than recorded
stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                channels = wf.getnchannels(),
                rate = wf.getframerate(),
                output = True)

# Read data in chunks
data = wf.readframes(chunk)

# Play the sound by writing the audio data to the stream
while data != '':
    stream.write(data)
    data = wf.readframes(chunk)

# Close and terminate the stream
stream.close()
p.terminate()

KeyboardInterrupt: 

Function for splitting up audio in large files and then transcribe as Google API only allows requests for up to 60 seconds per clip:

In [52]:
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [53]:
filepath = "SteveJobsCommencementSpeech.wav"
transcript = get_large_audio_transcription(filepath)

audio-chunks/chunk1.wav : Thank you. 
audio-chunks/chunk2.wav : Imma. 
audio-chunks/chunk3.wav : Honor to be with you today for your commencement from one of the finest universities in the world. 
Error: 
audio-chunks/chunk5.wav : Truth be told. 
audio-chunks/chunk6.wav : I never graduated from college. 
audio-chunks/chunk7.wav : Anna. 
audio-chunks/chunk8.wav : This is the closest i've ever gotten to a college graduation. 
audio-chunks/chunk9.wav : Today i want to tell you three stories from my life. 
audio-chunks/chunk10.wav : That's it no big deal. 
audio-chunks/chunk11.wav : Just three stories. 
audio-chunks/chunk12.wav : The first story. 
audio-chunks/chunk13.wav : It's about connecting the dots. 
audio-chunks/chunk14.wav : I dropped out of college after the first six months but then stayed around as a drop in for another 18 months or so before i really quit. 
audio-chunks/chunk15.wav : So why'd i drop out. 
audio-chunks/chunk16.wav : It started before i was born. 
audio-chunks/ch

audio-chunks/chunk109.wav : And it was devastating. 
audio-chunks/chunk110.wav : I really didn't know what to do for a few months. 
audio-chunks/chunk111.wav : I felt that i'd let the previous-generation of entrepreneurs down. 
audio-chunks/chunk112.wav : Did i drop the baton as it was being passed me. 
audio-chunks/chunk113.wav : I met with david packard and bob noyce. 
audio-chunks/chunk114.wav : And try to apologize for screwing up so badly. 
audio-chunks/chunk115.wav : I was a very public failure and i even thought about running away from the valley. 
audio-chunks/chunk116.wav : But something slowly began to dawn on me. 
audio-chunks/chunk117.wav : I still love. 
audio-chunks/chunk118.wav : What i did. 
audio-chunks/chunk119.wav : The turn of events at apple id not change that one bit. 
audio-chunks/chunk120.wav : I've been rejected. 
audio-chunks/chunk121.wav : But i was still in love. 
audio-chunks/chunk122.wav : And so i decided to start over. 
audio-chunks/chunk123.wav : I didn

audio-chunks/chunk218.wav : Your time is limited so don't waste it living someone else's life. 
audio-chunks/chunk219.wav : Don't be trapped by dogma which is living with the results of other people's thinking. 
audio-chunks/chunk220.wav : Don't let the noise of others opinions drown out your own inner voice. 
audio-chunks/chunk221.wav : And most important. 
audio-chunks/chunk222.wav : Have the courage to follow your heart and intuition. 
audio-chunks/chunk223.wav : They somehow already know. 
audio-chunks/chunk224.wav : What you truly want to become. 
audio-chunks/chunk225.wav : Everything else. 
audio-chunks/chunk226.wav : Is secondary. 
Error: 
audio-chunks/chunk228.wav : When i was young. 
audio-chunks/chunk229.wav : There was an amazing publication. 
audio-chunks/chunk230.wav : Call the whole earth catalog. 
audio-chunks/chunk231.wav : Which was one of the bible's of my generation. 
audio-chunks/chunk232.wav : It was created by a fellow named stewart brand not far from here in men

In [38]:
print("Steve Jobs Stanford Commencement Speech: \n")
print(transcript)

Steve Jobs Stanford Commencement Speech: 

Thank you. Imma. Honor to be with you today for your commencement from one of the finest universities in the world. Truth be told. I never graduated from college. Anna. This is the closest i've ever gotten to a college graduation. Today i want to tell you three stories from my life. That's it no big deal. Just three stories. The first story. It's about connecting the dots. I dropped out of college after the first six months but then stayed around as a drop in for another 18 months or so before i really quit. So why'd i drop out. It started before i was born. My biological mother. Was a young unwed graduate student. And she decided to put me up for adoption. She felt very strongly that i should be adopted by college graduates so everything was all set. For me to be adopted at birth by a lawyer and his wife. Accept it when i popped out. They decided at the last minute that they really wanted a girl. So my parents who are on a waiting list got a 

In [41]:
# Total words in transcript
print(f"Total words in transcript: {len(transcript)}")

Total words in transcript: 11873


In [45]:
# Loading official transcript from txt file
file_name = "steveJobsTranscript.txt"
official_transcript = ""

with open(file_name, 'rt') as fd:
    official_transcript = fd.readlines()

In [47]:
# Join lines as one entire string and then print number of words
official_transcript = "".join(official_transcript)

print(f"Total words in official transcript: {len(official_transcript)}")

Total words in official transcript: 11921


In [49]:
# Evaluate how many words lost during transcription
words_missed = len(official_transcript)-len(transcript)
print(f"Words missed from official transcript: {words_missed}")

percent_missed = (words_missed/len(official_transcript))*100
print(f"Percentage of missed words from total in official transcript: {percent_missed:.2f}%")

Words missed from official transcript: 48
Percentage of missed words from total in official transcript: 0.40%
